In [8]:
import transformers
import torch
import os 
from transformers import LlamaForCausalLM, LlamaTokenizer, BitsAndBytesConfig, AutoTokenizer
from peft import PeftModel

In [2]:
access_tok = '-'

In [3]:
default_path = os.getcwd()
data_path = os.path.join(default_path, '../../data')
model_path = os.path.join(default_path, '../../model')
config_path = os.path.join(default_path, '../../config')

In [4]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda', index=0)

In [43]:
generation_config = dict(
    temperature=1,
    top_k=40,
    top_p=0.9,
    do_sample=True,
    max_new_tokens=200,
    early_stopping=True,
    no_repeat_ngram_size=3,
    eos_token_id=2, 
    num_beams=1,
    repetition_penalty=1.1,
)

In [44]:
generation_config['num_beams']

1

In [45]:
def gen(x, config, model, tokenizer, device):
    prompt = (
        f"아래는 작업을 설명하는 명령어입니다. 요청을 적절히 완료하는 응답을 작성하세요.\n\n### 명령어:\n{x}\n\n### 응답:"
    )
    len_prompt = len(prompt)
    gened = model.generate(
        **tokenizer(prompt, return_tensors="pt", return_token_type_ids=False).to(
            device
        ),
        **config
    )
    return tokenizer.decode(gened[0])[len_prompt:]

In [46]:
model_id = "kfkas/Llama-2-ko-7b-Chat"

In [38]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = LlamaForCausalLM.from_pretrained(
    model_id, device_map={"": 0},torch_dtype=torch.float16, low_cpu_mem_usage=True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [39]:
tokenizer.pad_token = tokenizer.eos_token
model.config.use_cache = (True)
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(46336, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
 

In [47]:
user_input = '신용등급 알려줘'

In [48]:
output = gen(user_input, generation_config, model=model, tokenizer=tokenizer, device=device)
output

'한 신용등급은 현재 시점에서 제공하지 않으므로, 신용 등급과 관련한 질문을 다시 알려주시면 감사하겠습니다.</s>'

#### PEFT Models

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

config = AutoConfig.from_pretrained('meta-llama/Llama-2-7b-chat-hf', token=access_tok)